In [1]:
import numpy as np
import IPython
from pydub import AudioSegment
import wave

# 正弦波
def sine(freq, t):
  # freq: 周波数（数値）
  # t: 時刻（ベクトル）
  return np.sin(2.0*np.pi*freq*t)

# 三角波
def triangle(freq, t):
    return np.abs((2*t*freq-1/2)%2 - 1) * 2 -1  

# 矩形波
def square(freq, t):
    # [0-pi] -> 1, [pi-2pi] -> -1
    return (np.ceil(t * freq * 2) % 2) * 2 - 1

# 25%パルス波
def pulse_quarter(freq, t):
    return (np.ceil(t * freq * 4 - 1) % 4 == 0) * 2 - 1

# 12.5%パルス波
def pulse_eighth(freq, t):
    return (np.ceil(t * freq * 8 - 1) % 8 == 0) * 2 - 1

# 音楽再生
def play_music(music):
  rate = 44100
  return IPython.display.Audio(music, rate=rate, autoplay=True)

# 十二平均率で周波数を求める
def note_to_freq(notestr):
    if notestr == 'R':
       return 0
    key, octave = notestr[:-1], notestr[-1] #スライスで文字列と数字を分ける
    keys = ["ド", "ド#", "レ", "レ#", "ミ", "ファ", "ファ#", "ソ", "ソ#", "ラ", "ラ#", "シ"]
    return 27.5 * 2 ** (int(octave) + (keys.index(key)-9)/12)

# メロディ作成
def make_mml(mml, bpm, sw):
    rate = 44100
    duration = 60.0/bpm
    durationtmp = duration
    music = np.array([])
    for s in mml:
        if '+2/3' in s:
            duration = (duration/3)*2
            s = s[:-4]
        elif '+2' in s:
            duration = duration*2
            s = s[:-2]
        elif '+3/4' in s:
            duration = (duration/4)*3
            s = s[:-4]
        elif '+3/2' in s:
            duration = (duration/2)*3
            s = s[:-4]
        elif '+3' in s:
            duration = duration*3
            s = s[:-2]
        elif '+4' in s:
            duration = duration*4
            s = s[:-2]
        elif '+1/2' in s:
            duration = duration/2
            s = s[:-4]
        elif '+1/3' in s:
            duration = duration/3
            s = s[:-4]
        elif '+1/4' in s:
            duration = duration/4
            s = s[:-4]
        
        t = np.linspace(0, duration, int(rate * duration))
        f = note_to_freq(s)
        if sw == 1:
            x = sine(f, t)
        elif sw == 2:
            x = triangle(f, t)
        elif sw == 3:
            x = square(f, t)
        elif sw == 4:
            x = pulse_quarter(f, t)
        else:
            x = pulse_eighth(f, t)
        music = np.append(music, x)
        duration = durationtmp
    return music

# メロディ入力
def mml_input(bpm):
    sw = int(input('音色を選んでください(やわらかい音:1～5:歪んだ音)：'))
    mml = input('メロディを,で区切って入力してください（例:ド4,ド#4,レ4 Rは休符)：')
    mml=mml.split(',')
    mml.append('R')
    m = make_mml(mml, bpm, sw)
    return m

# 楽曲保存
def do_save(music, fname):
    sampling_rate = 44100
    # 16ビットで量子化
    music = (music*32767).astype(np.int16)
    with wave.Wave_write(fname) as fp:
        fp.setframerate(sampling_rate)
        fp.setnchannels(1) # モノラル
        fp.setsampwidth(2) # 16ビット（バイト数を入力する）
        fp.writeframes(music.tobytes()) # バイナリ化
    print('保存しました')

# 保存するか尋ねる
def save_input(music):
    save = input('保存しますか？Y/N:')
    if save == 'Y':
        fname = input('名前を付けてください：')
        fname = fname + '.wav'
        do_save(music, fname)
        to_mp3(fname)
    else:
        exit()
    
def to_mp3(fname):
    # 読み込み
    sound = AudioSegment.from_wav(fname) 
    rname = fname.rstrip('.wav') + '.mp3'
    # 変換
    sound.export(rname, format="mp3")
    return print('mp3に変換しました。')

def waon():
    q = input('和音を作成しますか？Y/N：')
    if q == 'Y':
        num = int(input('作成する和音の数を入力：'))
        for i in range(num):
            if i == 0:
                bpm = int(input('BPMを入力してください:'))
                music = mml_input(bpm)
            else:
                m = mml_input(bpm)
                music = music + m
    else:
        bpm = int(input('BPMを入力してください:'))
        music = mml_input(bpm)
    return music * 0.1 #修正①和音でも音声ファイルに保存できるように修正

In [ ]:
# 音、テンポ、文字列を入力
music = waon()
# 再生2
play_music(music)

In [ ]:
# 保存するか確認して、Yesの場合名前を付けて保存
save_input(music)